## Business problem

We have a sample of customer data from a bank. The bank has been seeing unusual churn rates and they want to understand the driving force behind this. The sample details customer data with a binary value as to whether the customer churned or not.

We need to create a segmentation model to determine the type of customer that is leaving the bank.

In [88]:
import numpy as np
import pandas as pd
import tensorflow as tf

## Data preprocessing

In [89]:
df = pd.read_csv('Churn_Modelling.csv')
df = df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [90]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

### Encoding categorical data

In [91]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [92]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

### Splitting the data into training and test

In [93]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Feature scaling

In [94]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Building the ANN

### Initializing the ANN

In [95]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

There is no specific number of neurons, we will need to experiment and tune the parameter with experience. Lets do an example using the Rectifier function.

In [96]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second layer

In [97]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

As we are dealing with a binary output, we can use 1 neuron. However, recall from our theory lesson that if we were dealing with categorical data in the output, we would need more.

In addition, we want to apply a sigmoid function on the output layer as we want to calculate the probability that the customer leaves the bank.

In [98]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training the ANN

### Compiling the ANN

In [99]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the ANN

We can set batch size to 32 for now but we can play with parameter to find what works for the data.

In [100]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 0s 668us/step - loss: 0.5473 - accuracy: 0.7721
Epoch 2/100
250/250 [==============================] - 0s 713us/step - loss: 0.4691 - accuracy: 0.7961
Epoch 3/100
250/250 [==============================] - 0s 824us/step - loss: 0.4457 - accuracy: 0.7983
Epoch 4/100
250/250 [==============================] - 0s 737us/step - loss: 0.4358 - accuracy: 0.8029
Epoch 5/100
250/250 [==============================] - 0s 705us/step - loss: 0.4306 - accuracy: 0.8089
Epoch 6/100
250/250 [==============================] - 0s 706us/step - loss: 0.4267 - accuracy: 0.8149
Epoch 7/100
250/250 [==============================] - 0s 739us/step - loss: 0.4226 - accuracy: 0.8185
Epoch 8/100
250/250 [==============================] - 0s 788us/step - loss: 0.4194 - accuracy: 0.8207
Epoch 9/100
250/250 [==============================] - 0s 716us/step - loss: 0.4159 - accuracy: 0.8239
Epoch 10/100
250/250 [==============================] - 0s 651us/step - l

250/250 [==============================] - 0s 664us/step - loss: 0.3320 - accuracy: 0.8660
Epoch 80/100
250/250 [==============================] - 0s 653us/step - loss: 0.3320 - accuracy: 0.8665
Epoch 81/100
250/250 [==============================] - 0s 684us/step - loss: 0.3318 - accuracy: 0.8649
Epoch 82/100
250/250 [==============================] - 0s 710us/step - loss: 0.3319 - accuracy: 0.8662
Epoch 83/100
250/250 [==============================] - 0s 673us/step - loss: 0.3314 - accuracy: 0.8658
Epoch 84/100
250/250 [==============================] - 0s 642us/step - loss: 0.3316 - accuracy: 0.8655
Epoch 85/100
250/250 [==============================] - 0s 642us/step - loss: 0.3316 - accuracy: 0.8649
Epoch 86/100
250/250 [==============================] - 0s 639us/step - loss: 0.3314 - accuracy: 0.8656
Epoch 87/100
250/250 [==============================] - 0s 643us/step - loss: 0.3319 - accuracy: 0.8651
Epoch 88/100
250/250 [==============================] - 0s 652us/step - loss:

We can see the accuracy improving over time and actually starts to converge after about 20 epochs.

## Making predicions and evaluating the model

### Predicting the result of a single observation

Remember to apply the transform method to avoid information leakage.

In [103]:
ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))

array([[0.03478527]], dtype=float32)

We can conclude that the customer is going to stay. Why? because using our sigmoid function, we have a probability of 0.034.. which will be classified as 0, i.e. does not leave the bank.

If we want to see the binary value, i.e. true or false, we can set the threshold to be 0.5.

In [104]:
ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5

array([[False]])

### Predicting the test results

In [105]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

np.set_printoptions(precision=2)
np.concatenate(
    (
        y_pred.reshape(len(y_pred), 1), 
        y_test.reshape(len(y_test), 1)
    ),
    1
)

array([[0, 0],
       [0, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 1],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [0, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 1],
       [0, 1],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 1],
       [0, 1],
       [1, 1],
       [0,

### Confusion matrix

In [108]:
from sklearn.metrics import confusion_matrix, accuracy_score

c_matrix = confusion_matrix(y_test, y_pred)
print(c_matrix)

acc = accuracy_score(y_test, y_pred)
print(acc)

[[1518   77]
 [ 201  204]]
0.861
